In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    print(training)
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("model created")

58 documents
13 classes ['about', 'complaint', 'connect', 'createaccount', 'explore', 'goodbye', 'greeting', 'help', 'name', 'random', 'startup', 'startups', 'thanks']
70 unique lemmatized words ["'s", 'a', 'about', 'account', 'all', 'an', 'anyone', 'are', 'asdf', 'bye', 'call', 'can', 'complaint', 'connect', 'could', 'create', 'do', 'explore', 'for', 'freelance', 'give', 'goodbye', 'hand', 'have', 'hay', 'hello', 'help', 'helpful', 'hey', 'hi', 'hiya', 'how', 'i', 'information', 'is', 'later', 'me', 'more', 'name', 'need', 'new', 'open', 'opportunity', 'page', 'people', 'please', 'qwerty', 'raise', 'requirement', 'sdfkwdlcm', 'see', 'service', 'should', 'some', 'start-ups', 'startup', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'to', 'want', 'what', 'whats', 'who', 'with', 'you', 'your']
[]
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

C:\Users\sreya\AppData\Local\Temp\ipykernel_15656\3728578087.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Epoch 1/200
12/12 [==============================] - 0s 1ms/step - loss: 2.6350 - accuracy: 0.0172
Epoch 2/200
12/12 [==============================] - 0s 2ms/step - loss: 2.5225 - accuracy: 0.0690
Epoch 3/200
12/12 [==============================] - 0s 2ms/step - loss: 2.4131 - accuracy: 0.2414
Epoch 4/200
12/12 [==============================] - 0s 1ms/step - loss: 2.3547 - accuracy: 0.2241
Epoch 5/200
12/12 [==============================] - 0s 1ms/step - loss: 2.2761 - accuracy: 0.3103
Epoch 6/200
12/12 [==============================] - 0s 3ms/step - loss: 2.1241 - accuracy: 0.3276
Epoch 7/200
12/12 [==============================] - 0s 1ms/step - loss: 1.9485 - accuracy: 0.4828
Epoch 8/200
12/12 [==============================] - 0s 1ms/step - loss: 2.0267 - accuracy: 0.3621
Epoch 9/200
12/12 [==============================] - 0s 2ms/step - loss: 1.7995 - accuracy: 0.5000
Epoch 10/200
12/12 [==============================] - 0s 1ms/step - loss: 1.6875 - accuracy: 0.4655
Epoch 11/

12/12 [==============================] - 0s 1ms/step - loss: 0.1903 - accuracy: 0.9483
Epoch 84/200
12/12 [==============================] - 0s 1ms/step - loss: 0.2488 - accuracy: 0.8793
Epoch 85/200
12/12 [==============================] - 0s 2ms/step - loss: 0.2401 - accuracy: 0.8966
Epoch 86/200
12/12 [==============================] - 0s 1ms/step - loss: 0.1864 - accuracy: 0.9310
Epoch 87/200
12/12 [==============================] - 0s 2ms/step - loss: 0.1851 - accuracy: 0.9310
Epoch 88/200
12/12 [==============================] - 0s 1ms/step - loss: 0.1865 - accuracy: 0.9138
Epoch 89/200
12/12 [==============================] - 0s 2ms/step - loss: 0.3466 - accuracy: 0.8276
Epoch 90/200
12/12 [==============================] - 0s 2ms/step - loss: 0.1724 - accuracy: 0.9138
Epoch 91/200
12/12 [==============================] - 0s 2ms/step - loss: 0.2320 - accuracy: 0.8966
Epoch 92/200
12/12 [==============================] - 0s 1ms/step - loss: 0.2065 - accuracy: 0.8966
Epoch 93/200


12/12 [==============================] - 0s 1ms/step - loss: 0.2319 - accuracy: 0.9483
Epoch 165/200
12/12 [==============================] - 0s 1ms/step - loss: 0.1521 - accuracy: 0.9310
Epoch 166/200
12/12 [==============================] - 0s 1ms/step - loss: 0.1353 - accuracy: 0.9310
Epoch 167/200
12/12 [==============================] - 0s 2ms/step - loss: 0.1688 - accuracy: 0.9138
Epoch 168/200
12/12 [==============================] - 0s 1ms/step - loss: 0.1680 - accuracy: 0.9138
Epoch 169/200
12/12 [==============================] - 0s 1ms/step - loss: 0.1273 - accuracy: 0.9138
Epoch 170/200
12/12 [==============================] - 0s 3ms/step - loss: 0.2114 - accuracy: 0.9138
Epoch 171/200
12/12 [==============================] - 0s 1ms/step - loss: 0.1459 - accuracy: 0.9138
Epoch 172/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1939 - accuracy: 0.9138
Epoch 173/200
12/12 [==============================] - 0s 2ms/step - loss: 0.1336 - accuracy: 0.9483
Epoc